In [ ]:
import os
import urllib.request
import json
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

from reportlab.pdfgen import canvas
from reportlab.platypus import (SimpleDocTemplate, Paragraph, PageBreak, Image, Spacer, Table, TableStyle, KeepInFrame, Frame)
from reportlab.lib.enums import TA_LEFT, TA_RIGHT, TA_CENTER, TA_JUSTIFY
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib.pagesizes import LETTER, inch
from reportlab.graphics.shapes import Line, LineShape, Drawing
from reportlab.lib.colors import Color
from reportlab.pdfbase.acroform import AcroForm
from reportlab.lib import colors
from reportlab.platypus.flowables import Flowable
import PIL

In [ ]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [ ]:
def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)

In [ ]:
def format_amount(indic, valeur):
    if "Montant" in indic:
        f_valeur = float(valeur)
        if f_valeur > 1000000:
            return str(round(f_valeur/1000000, 1)) + ' M€'
        elif f_valeur > 10000:
            return str(round(f_valeur/1000, 1)) + ' k€'
        else:
            return str(f_valeur)
    else:
        try:
            return int(valeur.split(".")[0])
        except ValueError as err:
            print(f"L'indicateur {indic} possède des valeurs invalides : {err}")

In [ ]:
mailles = ["national", "regional", "departemental"]

In [ ]:
pp_dep = pd.read_csv("pp_dep.csv", sep=";", dtype={"reg":str, "dep":str})

In [ ]:
list_mesure_indic = list(pp_dep.pivot_table(index=["short_mesure", "short_indic"], values="valeur").index)

In [ ]:
short_mesures_to_keep =set([
'AAP et AMI Efficacité énergétique',
'Assurance prospection',
"Ma Prime Rénov'",
"Prime à l'embauche des jeunes",
"Prime à l'embauche pour les travailleurs handicapés",
"Apprentissage",
"Bonus électrique",
"Contrats Initiatives Emploi (CIE) Jeunes",
'Contrats de professionnalisation',
'France Num : aide à la numérisation des TPE,PME,ETI',
'Garantie jeunes',
"AAP industrie : Modernisation des filières auto et aéro",
"Parcours emploi compétences (PEC) Jeunes",
'Prime à la conversion des véhicules légers',
"AAP Industrie : Soutien aux projets industriels territoires",
"AAP Industrie : Sécurisation approvisionnements critiques",
"Renforcement subventions Business France",
'Rénovation des bâtiments Etats (marchés notifiés)',
"Service civique",
'Soutien recherche aéronautique civil'
])
#print(list_mesure_indic)
#print(list(x[0] for x in list_mesure_indic if not(x[0] in short_mesures_to_keep)))
list_mesure_indic = [x for x in list_mesure_indic if x[0] in short_mesures_to_keep]

In [ ]:
pp_reg = pd.pivot_table(pp_dep, index=["mesure","short_mesure", "reg","region", "Date", "period_date", "short_indic"], values="valeur", aggfunc=np.sum)
pp_reg.rename(columns={"reg":"libelle"}, inplace=True)
pp_reg.reset_index(inplace=True)

In [ ]:
pp_nat = pd.pivot_table(pp_reg, index=["mesure", "short_mesure", "Date","period_date", "short_indic"], values="valeur", aggfunc=np.sum)
pp_nat.reset_index(inplace=True)

In [ ]:
dict_mesure_indic = {}
for x in list_mesure_indic:
    if x[0] in dict_mesure_indic:
        dict_mesure_indic[x[0]].append(x[1])
    else:
        dict_mesure_indic[x[0]] = [x[1]]

In [ ]:
dict_mesure_indic['AAP Industrie : Soutien aux projets industriels territoires'] = ['Nombre de TPE,PME,ETI bénéficiaires']
dict_mesure_indic['AAP Industrie : Sécurisation approvisionnements critiques'] = ['Nombre de TPE,PME,ETI bénéficiaires']
dict_mesure_indic['AAP et AMI Efficacité énergétique'] = ["Nombre d'entreprises ayant reçu l'aide"]
dict_mesure_indic['AAP industrie : Modernisation des filières auto et aéro'] = ['Nombre de PME']

In [ ]:
# On ne veut pas afficher les lignes de Prime Rénov nulles
pp_dep = pp_dep.loc[(pp_dep.short_mesure != "Ma Prime Rénov'") | (pp_dep.valeur != 0) ]

In [ ]:
pp_dep.valeur = pp_dep.valeur.astype(str)
pp_dep.valeur = pp_dep.apply(lambda x: format_amount(x["short_indic"], x["valeur"]), axis=1)

In [ ]:
pp_reg.valeur = pp_reg.valeur.astype(str)
pp_reg.valeur = pp_reg.apply(lambda x: format_amount(x["short_indic"], x["valeur"]), axis=1)

In [ ]:
pp_nat.valeur = pp_nat.valeur.astype(str)
pp_nat.valeur = pp_nat.apply(lambda x: format_amount(x["short_indic"], x["valeur"]), axis=1)

In [ ]:
taxo_dep_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/departements-minify.json'
taxo_reg_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/regions-minify.json'

In [ ]:
taxo_dep_df = pd.DataFrame(import_json_to_dict(taxo_dep_url))
dep_list = list(taxo_dep_df['dep'])
print('{} departements.'.format(len(dep_list)))

taxo_reg_df = pd.DataFrame(import_json_to_dict(taxo_reg_url))
reg_list = list(taxo_reg_df['reg'])
print('{} regions.'.format(len(reg_list)))

In [ ]:
#Building the folders structures
img_dir_path = './img/'
reports_dir_path = './reports/'
mkdir_ifnotexist(reports_dir_path)

In [ ]:
all_charts_as_df = {"departemental": {dep: {} for dep in dep_list},
                    "national": {'France': {}},
                    "regional": {reg: {} for reg in reg_list}}
    

In [ ]:
def make_pp_chart(maille, mesure, short_indics):
    if maille == "departemental":
        df = pp_dep.loc[(pp_dep.short_mesure == mesure)].sort_values(by="period_date", ascending=True).copy()
        deps = taxo_dep_df.dep.unique()  # Liste exhaustive de départements
        
        default = df.groupby(["Date", "period_date"]).sum().sort_values("period_date", ascending=True).reset_index()
        default[short_indics] = '-'
        default = default[["Date"] + short_indics]

        for dep in deps:
            print(f"Plotting {mesure}-{short_indics} : departement {dep}")
            df_dep = df.loc[df.dep == dep]
            if df_dep.shape[0] == 0:
                all_charts_as_df[maille][dep][mesure] = default.T.reset_index().T
            else:
                df_plot = pd.pivot_table(df_dep, index=['period_date', 'Date'], columns=['short_indic'], values='valeur', aggfunc='first')
                df_plot = df_plot.reset_index().drop('period_date', axis=1)
                df_plot = df_plot.rename_axis(None, axis=1)
                df_plot = df_plot.fillna('-')
                df_plot = df_plot[['Date'] + short_indics]
                all_charts_as_df[maille][dep][mesure] = df_plot.T.reset_index().T
                
            
    elif maille == "regional":
        df = pp_reg.loc[(pp_reg.short_mesure == mesure)].sort_values(by="period_date", ascending=True).copy()
        regs = taxo_dep_df.reg.unique()  # Liste exhaustive des régions
        
        default = df.groupby(["Date", "period_date"]).sum().sort_values("period_date", ascending=True).reset_index()
        default[short_indics] = '-'
        default = default[["Date"] + short_indics]

        for reg in regs:
            print(f"Plotting region {mesure}-{short_indics} : {reg}")
            df_reg = df.loc[df.reg == reg]
            if df_reg.shape[0] == 0:
                all_charts_as_df[maille][reg][mesure] = default.T.reset_index().T
            else:
                df_plot = pd.pivot_table(df_reg, index=['period_date', 'Date'], columns=['short_indic'], values='valeur', aggfunc='first')
                df_plot = df_plot.reset_index().drop('period_date', axis=1)
                df_plot = df_plot.rename_axis(None, axis=1)
                df_plot = df_plot.fillna('-')
                df_plot = df_plot[['Date'] + short_indics]
                all_charts_as_df[maille][reg][mesure] = df_plot.T.reset_index().T
            
    elif maille == "national":
        print(f"Plotting country {mesure}-{short_indics}")
        df_nat = pp_nat.loc[(pp_nat.short_mesure == mesure)].sort_values(by="period_date", ascending=True).copy()
        df_plot = pd.pivot_table(df_nat, index=['period_date', 'Date'], columns=['short_indic'], values='valeur', aggfunc='first')
        df_plot = df_plot.reset_index().drop('period_date', axis=1)
        df_plot = df_plot.rename_axis(None, axis=1)
        df_plot = df_plot.fillna('-')
        df_plot = df_plot[['Date'] + short_indics]
        all_charts_as_df[maille]['France'][mesure] = df_plot.T.reset_index().T

In [ ]:
def make_all_charts():
    for short_mesure in dict_mesure_indic:
        short_indics = dict_mesure_indic[short_mesure]
        for maille in mailles :
            make_pp_chart(maille, short_mesure, short_indics)


In [ ]:
make_all_charts()

In [ ]:
volet2mesures = {
'Ecologie': ["Ma Prime Rénov'",
		"Bonus électrique",
		'AAP et AMI Efficacité énergétique',
		'Prime à la conversion des véhicules légers',
		'Soutien recherche aéronautique civil', 
		'Rénovation des bâtiments Etats (marchés notifiés)',],

'Compétitivité': ['Assurance prospection', 
		'France Num : aide à la numérisation des TPE,PME,ETI',
		"AAP Industrie : Soutien aux projets industriels territoires",
		"AAP Industrie : Sécurisation approvisionnements critiques",
		"AAP industrie : Modernisation des filières auto et aéro",
		"Renforcement subventions Business France",],

'Cohésion': ["Apprentissage",
		"Prime à l'embauche des jeunes",
		"Prime à l'embauche pour les travailleurs handicapés",
		"Contrats Initiatives Emploi (CIE) Jeunes",
		'Contrats de professionnalisation',
		'Garantie jeunes',
		"Parcours emploi compétences (PEC) Jeunes",
		"Service civique",]

}

In [ ]:
def get_kpi(dep, short_indic, short_mesure):
    kpi_dep = (pp_dep.loc[(pp_dep.dep == dep) 
                          & (pp_dep.short_mesure == short_mesure) 
                          & (pp_dep.short_indic == short_indic)]
                .sort_values(by="period_date", ascending=False))
    if kpi_dep.shape[0] != 0:
        date= kpi_dep.iloc[0].Date
        valeur = kpi_dep.iloc[0].valeur
    else:
        date = pp_dep.Date.max()
        valeur = 0
    return date, valeur



class ReportCanvas(canvas.Canvas):

    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self.pages = []
        self.width, self.height = LETTER
        self.flag = False

    def showPage(self):
        self.pages.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        page_count = len(self.pages)
        for page in self.pages:
            self.__dict__.update(page)
            self.draw_canvas(page_count, self._pageNumber)
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_canvas(self, page_count, page_number):
        page = f"Page {self._pageNumber} sur {page_count}"
        x = 128
        self.saveState()
        self.setStrokeColorRGB(0, 0, 0)
        self.setLineWidth(0.5)
        self.drawImage("img/LogoRF.png", self.width - inch * 8 - 5, self.height - 120, width=100, height=100,
                       preserveAspectRatio=True)
        self.drawImage("img/logo_Francerelance_1321294.54.png", self.width - inch * 2, self.height - 130, width=130, height=130,
                       preserveAspectRatio=True)
        if page_number > 1:
            self.line(130, 680, LETTER[0] - 130, 680)  # Séparateur haut-de-page
            self.line(66, 78, LETTER[0] - 66, 78)  # Séparateur pied-de-page
            self.setFont('Times-Roman', 10)
            self.drawString(LETTER[0] - x, 65, page)
        self.restoreState()
        
        
class InteractiveTextfield(Flowable):
    def __init__(self, x, y, width, height, name='textfield'):
        Flowable.__init__(self)
        self.name = name
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def draw(self):
        self.canv.saveState()
        form = self.canv.acroForm
        form.textfield(name=self.name, relative=True, 
                       x=self.x, y=self.y, width=self.width, height=self.height,
                      maxlen=255, value='Entrez un commentaire...', fillColor=colors.white, )
        self.canv.restoreState()
        return

    
class PDFReport:
    
    def __init__(self, path, code_dep):
        self.path = path
        self.styleSheet = getSampleStyleSheet()
        self.elements = []  # Story
        self.dep = code_dep
        self.dep_name = taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['libelle']
        self.font = 'Arial'
        self.page_count = 0  # Increment 

    
    def create_front_page(self):
        # All texts
        text_title = "<b>SUIVI TERRITORIAL <br/> DU PLAN DE RELANCE</b>"
        text_subtitle = f"<i>Données pour le département : {self.dep_name}</i>"
        text_date = f"<i>Date : {datetime.datetime.today().strftime('%Y-%m-%d')}</i>"
        
        self.elements.append(Spacer(30, 150))  # Décale le titre vers le bas
        
        # Titre du rapport
        title_style = ParagraphStyle(self.font, fontSize=30, leading=60, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        title = Paragraph(text_title, title_style)
        self.elements.append(title)
        
        self.elements.append(Spacer(0, 30))
        
        # Sous-titre avec le nom du département
        subtitle_style = ParagraphStyle(self.font, fontSize=20, leading=50, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        subtitle = Paragraph(text_subtitle, subtitle_style)
        self.elements.append(subtitle)
        
        # Date de création du rapport
        date_style = ParagraphStyle(self.font, fontSize=20, leading=50, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        date = Paragraph(text_date, date_style)
        self.elements.append(date)
        
        self.elements.append(PageBreak())
        self.page_count += 1
        
        
    def create_volet_page(self, volet):
        # All texts
        text_title = f"<b>Volet : {volet}</b>"

        self.elements.append(Spacer(30, 150))  # Décale le titre vers le bas
        
        # Titre du rapport
        title_style = ParagraphStyle(self.font, fontSize=30, leading=60, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        title = Paragraph(text_title, title_style)
        self.elements.append(title)
        
        self.elements.append(PageBreak())
        self.page_count += 1
    
    
    def create_content_page(self, short_mesure, volet):
        reg = taxo_dep_df[taxo_dep_df['dep'] == self.dep].iloc[0]['reg']
        reg_name = taxo_reg_df[taxo_reg_df['reg'] == reg].iloc[0]['libelle']
        
        # Affichage mesure
        text_mesure = f"<b>{short_mesure}</b>"
        mesure_style = ParagraphStyle(self.font, fontSize=15, leading=15, justifyBreaks=1, alignment=TA_CENTER, justifyLastLine=1)
        mesure = Paragraph(text_mesure, mesure_style)
        block_mesure = KeepInFrame(288, 576, [mesure], hAlign='CENTER')  # Pour créer des retours à la ligne en cas de textes longs
        self.elements.append(block_mesure)
        self.elements.append(Spacer(1, 60))
        
        # Affichage phrase indic
        short_indic = dict_mesure_indic[short_mesure][0]
        date, valeur = get_kpi(self.dep, short_indic, short_mesure)
        text_indic = f"En {date}, {short_indic} : {valeur} (données cumulées)"
        indic_style = ParagraphStyle(self.font, fontSize=11, leading=20, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        sentence_indic = Paragraph(text_indic, indic_style)
        self.elements.append(sentence_indic)
        
        # Affichage par niveau de localité --> Table
        text_national = f"Niveau National"
        df_mesure_national = all_charts_as_df['national']['France'][short_mesure]
        title_national, table_national = self._get_block_indic_for_locality(text_national, df_mesure_national)
        
        text_regional = f"Niveau Régional : {reg_name}"
        df_mesure_regional = all_charts_as_df['regional'][reg][short_mesure]
        title_regional, table_regional = self._get_block_indic_for_locality(text_regional, df_mesure_regional)
        
        text_departemental = f"Niveau Départemental : {self.dep_name}"
        df_mesure_departemental = all_charts_as_df['departemental'][dep][short_mesure]
        title_departemental, table_departemental = self._get_block_indic_for_locality(text_departemental, df_mesure_departemental)
    
        data_table = [  [text_national,],
                        [table_national, ],
                        [text_regional, ],
                        [table_regional, ],
                        [text_departemental, ],
                        [table_departemental,],]

        indic_cell_height = 130
        title_cell_height = 20
        row_table_heights = [title_cell_height, indic_cell_height, 
                             title_cell_height, indic_cell_height, 
                             title_cell_height, indic_cell_height, 
                            ]
        style = TableStyle(
            [
                ('FONTNAME', (0,0), (0,-1), 'Helvetica-Bold'),
                ('ALIGN', (0,0), (0,-1), 'CENTER'),
                ('VALIGN',(0,0),(0,-1),'TOP'),
            ]
        )
        table = Table(data_table, colWidths=10, rowHeights=row_table_heights, hAlign='CENTER', style=style)
        self.elements.append(table)
        
        form_textfield = InteractiveTextfield(x=0, y=-10, width=LETTER[0]-132, height=50, name=f"textfield{self.page_count}")
        self.elements.append(form_textfield)

        self.elements.append(PageBreak())
        self.page_count += 1
        
    def _get_block_indic_for_locality(self, text_title, df_indics):
        # Pour l'affichage titre
        title = Paragraph(text_title)

        # Pour l'affichage de la table
        blue_header = colors.Color(red=0,green=0,blue=0.3, alpha=0.75)
        grey_color = colors.Color(red=0, green=0, blue=0, alpha=0.1)
        
        # Style pour paragraphe
        style_header = ParagraphStyle('header', fontSize=10, wordWrap='CJK', alignment=TA_CENTER, fontName='Helvetica-Bold')
        style_content = ParagraphStyle('content', fontSize=10, alignment=TA_CENTER)
        text_table_content = []
        for i, row in enumerate(df_indics.values):
            text_table_content.append([])
            for cell in row:
                if i == 0:
                    text_table_content[-1].append(Paragraph(f'<font color={colors.white}>{cell}</font>', style_header))
                else:
                    text_table_content[-1].append(Paragraph(str(cell), style_content))
                
        # Style pour tableau
        table_style = [
            ('BACKGROUND', (0,0), (-1,0), blue_header),  # Header
            ('VALIGN', (0,0), (-1,-1), 'TOP')
        ]
        for i in range(1, df_indics.shape[0]):
            if i % 2 == 0:
                table_style.append(('BACKGROUND', (0,i), (-1,i), grey_color))
            else:
                table_style.append(('BACKGROUND', (0,i), (-1,i), colors.white),)
        table_style = TableStyle(table_style)
        table = Table(text_table_content, colWidths=120, style=table_style)
        return title, table

    def save_report(self):
        self.doc = SimpleDocTemplate(self.path, pagesize=LETTER, topMargin=50)
        self.doc.multiBuild(self.elements, canvasmaker=ReportCanvas)


def create_single_report(dep, report_path):
    """ Créer un document PDF concernant le departement indiqué.
    :param dep code département (ex: 01, 72...)
    :param report_path
    """
    report = PDFReport(report_path, dep)
    
    report.create_front_page()

    for volet, short_mesures in volet2mesures.items():
        report.create_volet_page(volet)
        for short_mesure in sorted(short_mesures):
            report.create_content_page(short_mesure, volet)

    report.save_report()


# TEST
#dep = '976'
#report_path = 'test_report.pdf'
#create_single_report(dep, report_path)

In [ ]:
mkdir_ifnotexist(os.path.join(reports_dir_path))

for dep in taxo_dep_df.dep.unique():
    departement = taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['libelle']
    report_path = os.path.join(reports_dir_path, 'Suivi_territorial_plan_relance_'+ departement+'.pdf')
    create_single_report(dep, report_path)
    
    print(str(datetime.datetime.today()) + ' - ' + dep + ' done.')